In [1]:
!nvidia-smi

Sat Apr 15 18:43:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Please restart runtime if [PIL] is already installed
  - Should only have to do it once

In [1]:
# Riffusion dependencies
!pip install pillow==9.1.0
!pip install accelerate
!pip install argh
!pip install dacite
!pip install demucs
!pip install diffusers>=0.9.0
!pip install flask
!pip install flask_cors
!pip install numpy
!pip install plotly
!pip install pydub
!pip install pysoundfile
!pip install scipy
!pip install soundfile
!pip install sox
!pip install streamlit>=1.10.0
!pip install torch
!pip install torchaudio
!pip install torchvision
!pip install transformers

     ---------------------------------------- 3.3/3.3 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
riffusion 0.0.5 requires pillow<10.0.0,>=9.4.0, but you have pillow 9.1.0 which is incompatible.


In [2]:
!git clone https://github.com/hmartiro/riffusion-inference
%cd riffusion-inference

c:\Users\kpk16\OneDrive\Desktop\cnjustin\riffusion-inference


Cloning into 'riffusion-inference'...


In [3]:
from pathlib import Path
from typing import Union, Optional
from os import getcwd, sep
import io
import sys

import PIL

sys.path.append(getcwd() + sep + "riffusion")

from riffusion.datatypes import InferenceInput, InferenceOutput
from riffusion.riffusion_pipeline import RiffusionPipeline
from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_params import SpectrogramParams
from riffusion.server import SEED_IMAGES_DIR



pipeline: RiffusionPipeline


def startup(
        checkpoint: str = "riffusion/riffusion-model-v1",
        no_traced_unet: bool = False,
        device: str = "cuda",
            ) -> None:
    """
    Initializes the pipeline.
    
    Parameters:
        checkpoint (str): a string of where to load a pretrained
        model.
        no_traced_unet (bool): whether to not use a traced unet
        for speedups.
        device (str): "cuda", "cpu", or "mps".

    Returns:
        None
    """
    global pipeline
    pipeline = RiffusionPipeline.load_checkpoint(
        checkpoint=checkpoint,
        use_traced_unet=not no_traced_unet,
        device=device,
    )

def compute(inputs: InferenceInput) -> InferenceOutput:
    """
    Function from the riffusion server :func:`compute_request`.
    
    Parameters:
        inputs (:py:class:`InferenceInput`): The inputs for the request.
    
    Returns:
        str: an :py:class:`InferenceOutput`.
    """
    # Load the seed image by ID
    init_image_path = Path(SEED_IMAGES_DIR, f"{inputs.seed_image_id}.png")

    if not init_image_path.is_file():
        return f"Invalid seed image: {inputs.seed_image_id}", 400
    init_image = PIL.Image.open(str(init_image_path)).convert("RGB")

    # Load the mask image by ID
    mask_image: Optional[PIL.Image.Image] = None
    if inputs.mask_image_id:
        mask_image_path = Path(SEED_IMAGES_DIR, f"{inputs.mask_image_id}.png")
        if not mask_image_path.is_file():
            return f"Invalid mask image: {inputs.mask_image_id}", 400
        mask_image = PIL.Image.open(str(mask_image_path)).convert("RGB")

    # Execute the model to get the spectrogram image
    image = pipeline.riffuse(
        inputs,
        init_image=init_image,
        mask_image=mask_image,
    )

    # TODO(hayk): Change the frequency range to [20, 20k] once the model is retrained
    params = SpectrogramParams(
        min_frequency=0,
        max_frequency=10000,
    )

    # Reconstruct audio from the image
    # TODO(hayk): It may help performance a bit to cache this object
    converter = SpectrogramImageConverter(params=params, device=str(pipeline.device))

    segment = converter.audio_from_spectrogram_image(
        image,
        apply_filters=True,
    )
    return segment

c:\Users\kpk16\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kpk16\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
startup(device="cuda")

C:\Users\kpk16\AppData\Local\Temp\ipykernel_5124\245974814.py:41: UserWarning: WARNING: cuda is not available, using cpu instead.
  pipeline = RiffusionPipeline.load_checkpoint(


c:\Users\kpk16\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kpk16\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]































































c:\Users\kpk16\AppData\Local\Programs\Python\Python310\lib\site-packages\diffusers\models\unet_2d_condition.py:452: FutureWarning: Accessing `in_channels` directly via unet.in_channels is deprecated. Please use `unet.config.in_channels` instead
  deprecate(


In [5]:
import base64
import json
from riffusion.datatypes import InferenceInput, PromptInput, InferenceOutput

seed: int = 1

def muse(prompt_start_input, prompt_end_input, alpha_input, num_steps: int = 50):    
    segment = museHelp(prompt_start_input, prompt_end_input, alpha_input, num_steps)

    # Export audio to MP3 bytes
    wav_bytes = io.BytesIO()
    segment.export(wav_bytes, format="wav")
    wav_bytes.seek(0)

    with open("output.mp3", "wb") as f:
        f.write(wav_bytes)
    return "output.mp3"

def museHelp(prompt_start_input, prompt_end_input, num_steps: int = 50):
    global seed
    segments = None
    for alpha_input in range(0, 4):
      ai: float = float(alpha_input) / 4
      transition: bool = prompt_start_input != prompt_end_input
      promptStart: PromptInput = PromptInput(
          prompt=prompt_start_input,
          seed=seed,
      )
      seed += 1 if transition else 0
      promptEnd: PromptInput = PromptInput(
          prompt=prompt_end_input,
          seed=seed,
      )
      inputTotal: InferenceInput = InferenceInput(
          start=promptStart,
          end=promptEnd,
          alpha=0.0,
          num_inference_steps=num_steps,
      )

      segment = compute(inputs=inputTotal)
      if segments == None:
        segments = segment
      else:
        segments += segment
      seed += 1
    return segments


In [6]:
from IPython.display import Audio
from IPython.display import display

promptStart = "classical piano" # @param {type: "string"}
promptEnd = "heavy metal" # @param {type: "string"}

segment  = museHelp(promptStart, promptEnd)
#filename = muse(promptStart, promptEnd, alpha)
wn = Audio(data=segment.get_array_of_samples(), rate=segment.frame_rate, autoplay=True)
display(wn)

 16%|█▌        | 6/38 [01:29<07:45, 14.53s/it]

In [ ]:
def convert(d: dict)-> bytes:
  return bytes(str(d), "utf-8")

In [ ]:
import socket, os
import pydub

server_ip: str = '3.141.210.37' # @param {type: "string"}
port: int = 11553 # @param {type: "number"}
CHUNK: int = 1024
sample_rate: int = 44100
client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
socket_address = (server_ip,port)
client_socket.connect(socket_address)
client: dict = {"client": "music", "threads": "1", "sessionNum": "0"}
client_socket.sendall(bytes(str(client), "utf-8"))

In [ ]:
import ast

def deconvert(message: bytes) -> Union[str, dict]:
   strMessage: str = message.decode("utf-8")
   try:
      dictMessage: dict = ast.literal_eval(strMessage)
      return dictMessage
   except ValueError:
      return strMessage
   return strMessage

In [ ]:
client_socket.recv

In [ ]:
def sendAudio(segment: pydub.AudioSegment):
  data = segment.get_array_of_samples()
  for i in range(0, len(data) + CHUNK, CHUNK):
    mini = data[i:min(i + CHUNK, len(data) - 1)]
    message: dict = {"data": mini, "sample_rate": segment.frame_rate, "end": False}
    client_socket.sendall(convert(message))
  client_socket.sendall(convert({"end": True}))

In [ ]:
def receive():
  stop = False
  while not stop:
    messages = client_socket.recv(CHUNK)
    inputs = deconvert(messages)
    if type(inputs) == str:
      print("AAH")
      skip = False
    else:
      if inputs["promptStart"] == "endstart" and inputs["promptEnd"] == "endend":
        stop = True
        continue
      music = museHelp(inputs["promptStart"], inputs["promptEnd"])
      sendAudio(music)

In [ ]:
receive()

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│ ▲                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: unexpected EOF while parsing

In [ ]:
client_socket.close()